In [1]:
import utilities
import os
import json
import statistics
from prettytable import PrettyTable
from prettytable import ALL as ALL
# from scipy.stats import ttest_ind as ttest

In [58]:
DATA_DIR = '/Users/umariqbal/Documents/voice-assistant-central/data' 

# CONTROL_PATH = os.path.join(DATA_DIR, 'control_persona/website_bids/')
# TEST_PATH = os.path.join(DATA_DIR, 'healthfitness_persona/website_bids/')

CONTROL_PATH = os.path.join(DATA_DIR, 'no-amazon/')
TEST_PATH = os.path.join(DATA_DIR, 'amazon-plain/')

no_amazon = '/Users/umariqbal/Documents/voice-assistant-central/data/no-amazon/'
amazon = '/Users/umariqbal/Documents/voice-assistant-central/data/amazon-plain/'
CONTROL_PATH = '/Users/umariqbal/Documents/voice-assistant-central/data/amazon-echo/'
CONTROL_PATH = '/Users/umariqbal/Documents/voice-assistant-central/data/health-fitness-new/'
TEST_PATH = '/Users/umariqbal/Documents/voice-assistant-central/data/smart-home/'

In [27]:
common_websites = set(get_websites_with_bids(utilities.get_files_in_a_directory(no_amazon))).intersection(set(get_websites_with_bids(utilities.get_files_in_a_directory(amazon))))\
.intersection(set(get_websites_with_bids(utilities.get_files_in_a_directory(amazon_echo)))).intersection(set(get_websites_with_bids(utilities.get_files_in_a_directory(health_fitness))))\
.intersection(set(get_websites_with_bids(utilities.get_files_in_a_directory(smart_home))))

In [3]:
PERSONA = 'amazon-account'

### Reading bids for personas

In [59]:
control_bid_files = utilities.get_files_in_a_directory(CONTROL_PATH)
test_bid_files = utilities.get_files_in_a_directory(TEST_PATH)

In [5]:
def get_forced_noraml_websites_with_bids(bid_files):
    forced_count = 0
    normal_count = 0
    for bid_file in bid_files:
        if 'FORCED' in bid_file:
            forced_count += 1
        else:
            normal_count += 1

    return normal_count, forced_count


def get_websites_with_bids(bid_files):
    websites_with_bids = []
    for bid_file in bid_files:
        websites_with_bids.append(bid_file.rsplit('/', 1)[1].rsplit('_', 1)[1])

    return websites_with_bids


def return_bid_cpms(bid_files):
    bid_cpms = []
    for bid_file in bid_files:
        all_bids = utilities.read_file(bid_file)

        for bid in all_bids:
            bid_json = json.loads(bid)
            bid_cpms.append(bid_json['cpm'])
    
    return bid_cpms


def return_bid_cpms_for_websites(bid_files, websites):
    bid_cpms = []
    for bid_file in bid_files:

        bid_website = bid_file.rsplit('/', 1)[1].rsplit('_', 1)[1]
        if bid_website not in websites:
            continue
            
        all_bids = utilities.read_file(bid_file)

        for bid in all_bids:
            bid_json = json.loads(bid)
            bid_cpms.append(bid_json['cpm'])
    
    return bid_cpms


def return_median_cpms(bid_files):
    bid_cpms = []
    for bid_file in bid_files:
        all_bids = utilities.read_file(bid_file)
        
        current_bid_cpms = []
        for bid in all_bids:
            bid_json = json.loads(bid)
            current_bid_cpms.append(bid_json['cpm'])
            
        bid_cpms.append(statistics.median(current_bid_cpms))
    
    return bid_cpms


def return_median_cpms_with_websites(bid_files, websites):
    bid_cpms = []
    for bid_file in bid_files:
        all_bids = utilities.read_file(bid_file)
        
        bid_website = bid_file.rsplit('/', 1)[1].rsplit('_', 1)[1]
        if bid_website not in websites:
            continue
            
        current_bid_cpms = []
        for bid in all_bids:
            bid_json = json.loads(bid)
            current_bid_cpms.append(bid_json['cpm'])
            
        bid_cpms.append(statistics.median(current_bid_cpms))
    
    return bid_cpms

In [6]:
def print_bid_statistics(control_cpms, test_cpms, persona, control_normal, control_forced,
                         test_normal, test_forced, control_websites, test_websites, 
                         common_websites, count_limit = 20):
    control_median = statistics.median(control_cpms)
    test_median = statistics.median(test_cpms)
    
    control_mean = "{0:0.3f}".format(statistics.mean(control_cpms))
    test_mean = "{0:0.3f}".format(statistics.mean(test_cpms))
    
    control_min = min(control_cpms)
    control_max = "{0:0.3f}".format(max(control_cpms))
    
    test_min = min(test_cpms)
    test_max = "{0:0.3f}".format(max(test_cpms))
    
    control_total_bids = len(control_cpms)
    test_total_bids = len(test_cpms)
    
    control_total_websites = len(control_websites)
    test_total_websites = len(test_websites)
    
    control_sd = "{0:0.3f}".format(statistics.stdev(control_cpms))
    test_sd = "{0:0.3f}".format(statistics.stdev(test_cpms))
    
    common_websites_len = len(common_websites)
    
    table = PrettyTable(['Measure', 'Control', persona], hrules=ALL)
    
    
    table.add_row(['# of websites', control_total_websites, test_total_websites])
    table.add_row(['# of bids', control_total_bids, test_total_bids])
    table.add_row(['Common websites', common_websites_len, common_websites_len])
    table.add_row(['Normal/Forced', str(control_normal) + '/'+ str(control_forced), str(test_normal) + '/'+ str(test_forced)])
    
    table.add_row(['Median', control_median, test_median])
    table.add_row(['Mean', control_mean, test_mean])
    table.add_row(['SD', control_sd, test_sd])
    table.add_row(['Min', control_min, test_min])
    table.add_row(['Max', control_max, test_max])
    
    print(table)

### Get websites with bids

In [60]:
control_websites = get_websites_with_bids(control_bid_files)
test_websites = get_websites_with_bids(test_bid_files)

In [7]:
common_websites = set(control_websites).intersection(set(test_websites))

In [61]:
control_normal_count, control_forced_count = get_forced_noraml_websites_with_bids(control_bid_files)
test_normal_count, test_forced_count = get_forced_noraml_websites_with_bids(test_bid_files)

In [62]:
control_normal_count, control_forced_count, test_normal_count, test_forced_count

(26, 7, 23, 7)

### Consider all websites and all bids

In [63]:
control_cpms = return_bid_cpms(control_bid_files)
test_cpms = return_bid_cpms(test_bid_files)

In [64]:
print_bid_statistics(control_cpms, test_cpms, PERSONA,
                     control_normal_count, control_forced_count, test_normal_count, test_forced_count,
                     control_websites, test_websites, common_websites)

+-----------------+---------+----------------+
|     Measure     | Control | amazon-account |
+-----------------+---------+----------------+
|  # of websites  |    33   |       30       |
+-----------------+---------+----------------+
|    # of bids    |   136   |      111       |
+-----------------+---------+----------------+
| Common websites |    14   |       14       |
+-----------------+---------+----------------+
|  Normal/Forced  |   26/7  |      23/7      |
+-----------------+---------+----------------+
|      Median     |  0.095  |      0.12      |
+-----------------+---------+----------------+
|       Mean      |  0.460  |     0.422      |
+-----------------+---------+----------------+
|        SD       |  1.004  |     0.942      |
+-----------------+---------+----------------+
|       Min       |    0    |       0        |
+-----------------+---------+----------------+
|       Max       |  5.480  |     5.480      |
+-----------------+---------+----------------+


### Consider bids on websites where bids returned for both personas

In [65]:
common_control_cpms = return_bid_cpms_for_websites(control_bid_files, common_websites)
common_test_cpms = return_bid_cpms_for_websites(test_bid_files, common_websites)

In [ ]:

ttest(common_control_cpms, common_test_cpms)

In [ ]:
# control_cpms, test_cpms

In [66]:
print_bid_statistics(common_control_cpms, common_test_cpms, PERSONA, 
                     control_normal_count, control_forced_count, test_normal_count, test_forced_count,
                     control_websites, test_websites, common_websites)

+-----------------+---------+----------------+
|     Measure     | Control | amazon-account |
+-----------------+---------+----------------+
|  # of websites  |    33   |       30       |
+-----------------+---------+----------------+
|    # of bids    |    52   |       46       |
+-----------------+---------+----------------+
| Common websites |    14   |       14       |
+-----------------+---------+----------------+
|  Normal/Forced  |   26/7  |      23/7      |
+-----------------+---------+----------------+
|      Median     |   0.12  |   0.1738925    |
+-----------------+---------+----------------+
|       Mean      |  0.342  |     0.379      |
+-----------------+---------+----------------+
|        SD       |  0.527  |     0.473      |
+-----------------+---------+----------------+
|       Min       |   0.01  |      0.01      |
+-----------------+---------+----------------+
|       Max       |  2.710  |     2.240      |
+-----------------+---------+----------------+


### Consider medain bids on websites 

In [67]:
median_control_cpms = return_median_cpms(control_bid_files)
median_test_cpms = return_median_cpms(test_bid_files)

In [68]:
print_bid_statistics(median_control_cpms, median_test_cpms, PERSONA, 
                     control_normal_count, control_forced_count, test_normal_count, test_forced_count, 
                     control_websites, test_websites, common_websites)

+-----------------+---------+----------------+
|     Measure     | Control | amazon-account |
+-----------------+---------+----------------+
|  # of websites  |    33   |       30       |
+-----------------+---------+----------------+
|    # of bids    |    33   |       30       |
+-----------------+---------+----------------+
| Common websites |    14   |       14       |
+-----------------+---------+----------------+
|  Normal/Forced  |   26/7  |      23/7      |
+-----------------+---------+----------------+
|      Median     |   0.11  |     0.1225     |
+-----------------+---------+----------------+
|       Mean      |  0.452  |     0.546      |
+-----------------+---------+----------------+
|        SD       |  1.021  |     1.099      |
+-----------------+---------+----------------+
|       Min       |   0.0   |       0        |
+-----------------+---------+----------------+
|       Max       |  5.480  |     5.480      |
+-----------------+---------+----------------+


### Consider medain bids on websites where bids returned for both personas

In [69]:
common_median_control_cpms = return_median_cpms_with_websites(control_bid_files, common_websites)
common_median_test_cpms = return_median_cpms_with_websites(test_bid_files, common_websites)

In [70]:
print_bid_statistics(common_median_control_cpms, common_median_test_cpms, PERSONA, 
                     control_normal_count, control_forced_count, test_normal_count, test_forced_count,  
                     control_websites, test_websites, common_websites)

+-----------------+---------+----------------+
|     Measure     | Control | amazon-account |
+-----------------+---------+----------------+
|  # of websites  |    33   |       30       |
+-----------------+---------+----------------+
|    # of bids    |    14   |       14       |
+-----------------+---------+----------------+
| Common websites |    14   |       14       |
+-----------------+---------+----------------+
|  Normal/Forced  |   26/7  |      23/7      |
+-----------------+---------+----------------+
|      Median     |  0.195  |     0.184      |
+-----------------+---------+----------------+
|       Mean      |  0.335  |     0.514      |
+-----------------+---------+----------------+
|        SD       |  0.411  |     0.742      |
+-----------------+---------+----------------+
|       Min       |   0.04  |      0.01      |
+-----------------+---------+----------------+
|       Max       |  1.555  |     2.240      |
+-----------------+---------+----------------+


In [28]:
common_websites

{'accuweather.com',
 'apnews.com',
 'arstechnica.com',
 'cnet.com',
 'digg.com',
 'espn.com',
 'howstuffworks.com',
 'investopedia.com',
 'metropoles.com',
 'theatlantic.com',
 'theguardian.com',
 'timeanddate.com',
 'wired.com',
 'zdnet.com'}

In [ ]:
for bid_file in test_bid_files:
    all_bids = utilities.read_file(bid_file)

    for bid in all_bids:
        bid_json = json.loads(bid)
        print(bid_json)
        break
    break